# Project Alternative Assignment 1 Template

## Goal: Diffuse and mirror reflection due to environmentlight.

## Imports

In [7]:
import pygame
import moderngl
import numpy
import glm
from loadModelUsingAssimp_V2 import create3DAssimpObject

pygame 2.6.1 (SDL 2.28.4, Python 3.13.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/rafa/Developer/GRAPHICS/.cgenv/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


## Intialize

#### Enable anti-aliasing in the GPU pipeline
 - At the time of context creation, request a multisample buffer.: pygame.display.gl_set_attribute(GL_MULTISAMPLEBUFFERS, 1).
 - Specify the number of samples per pixel for multisampling: pygame.display.gl_set_attribute(GL_MULTISAMPLESAMPLES, 16). Higher the number better the antialiasing. Check the max number of samples supported in your driver. (ctx.max_samples)

In [8]:
width = 840
height = 480

pygame.init() # Initlizes its different modules. Display module is one of them.
pygame.display.gl_set_attribute(pygame.GL_MULTISAMPLEBUFFERS, 1)
pygame.display.gl_set_attribute(pygame.GL_MULTISAMPLESAMPLES, 16)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE) 
pygame.display.set_mode((width, height), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title = "Class Practice: Instructor")
gl = moderngl.get_context() # Get Previously created context.
gl.info["GL_VERSION"]

'4.6 (Compatibility Profile) Mesa 25.2.5-arch1.2'

## Reflecting Object: 
Uses a Dummy Fragment shader code.  
It must be modified for computing lighting due to environment light. See the assignment details.

In [2]:
#
# Vertex shader(s)
#
vertex_shader_code = '''
#version 460 core
layout (location=0) in vec3 position;
layout (location=1) in vec3 normal;
layout (location=2) in vec2 uv;


uniform mat4 model, view, perspective;
uniform mat3 normalMatrix;

out vec2 f_uv;
out vec3 f_normal; // Normal vector in World Coordinates
out vec3 f_position; // postion in world coordinates
void main() {
    f_uv = uv;
    vec4 P = model*vec4(position, 1);
    f_position = P.xyz;j
    gl_Position = perspective*view*P;
    mat3 normalMatrix = mat3(transpose(inverse(model)));// inverse transpose of model transformation
    f_normal = normalize(normalMatrix*normal);
}
'''

#
# Fragment shader(s)
#
fragment_shader_code = '''
#version 460 core

in vec3 f_normal;
in vec3 f_position;
in vec2 f_uv;

uniform sampler2D map;
uniform vec3 light;
uniform vec3 eye_position;
uniform bool metal;

const float shininess = 5;

// Add output variable here
out vec4 out_color;

vec3 computeColor(){
    vec3 color = vec3(0);
    vec3 N = normalize(f_normal);
    vec3 V = normalize(eye_position - f_position);
    vec3 L = normalize(light);
    vec3 H = normalize(L + V);
    vec3 materialColor = texture(map, f_uv).rgb;
    if (dot(N, L) > 0.){
        if (metal)
            color = materialColor*pow(dot(N,H), shininess);
        else color = materialColor*dot(N,L);
    }
    return color;
}
void main() {
    out_color = vec4(computeColor(), 1);
}
'''
#
# Programs
#

model_program = gl.program(
    vertex_shader= vertex_shader_code,
    fragment_shader= fragment_shader_code
)
format = "3f 3f 2f"
variables = ["position","normal", "uv"]

### Use Assimp parser and and read a 3D model.
modelFile = "the_utah_teapot/scene.gltf" 
modelObj = create3DAssimpObject(modelFile, verbose=False, textureFlag = True, normalFlag = True)

model_renderables = modelObj.getRenderables(gl, model_program, format, variables)
scene = modelObj.scene

#### Recursive Render for scenegraph

def recursive_render(node, M):
    nodeTransform = glm.transpose(glm.mat4(node.transformation));
    currentTransform = M * nodeTransform
    if node.num_meshes > 0:
        for index in node.mesh_indices:
            model_renderables[index]._program["model"].write(currentTransform)         
            model_renderables[index].render()
            
    for node in node.children:
        recursive_render(node, currentTransform)

def render():
    recursive_render(scene.root_node, M=glm.mat4(1))
    
_imageFile = "gold.jpg"
_texture_img = pygame.image.load(_imageFile) 
_texture_data = pygame.image.tobytes(_texture_img,"RGB", True) 
_texture = gl.texture(_texture_img.get_size(), data = _texture_data, components=3)
gold_sampler = gl.sampler(texture=_texture)
bound = modelObj.bound

NameError: name 'gl' is not defined

### Skybox rendering template: 
Uses a dummy vertex shader and a dummy fragment shader.  
It must be modified to carry out skybox rendering.

In [ ]:
_positions = numpy.array([
    [-1, 1],
    [ 1, 1],
    [ 1,-1],
    [-1,-1]
]).astype("float32")

_geom = _positions.flatten()

_index = numpy.array([
    0, 1, 2,
    2, 3, 0
]).astype("int32")

#
# Vertex shader(s)
#
_vertex_shader_code = '''
#version 460 core
in vec2 position;

void main() {
    gl_Position = vec4(position,1,1);
 }
'''
#
# Fragment shader(s)
#
_fragment_shader_code = '''
#version 460 core
in vec3 f_position;

uniform vec3 eye_position;

// Add output variable here
out vec4 out_color;

void main() {
    out_color = vec4(0.5,0.5,0.5,1);
}
'''
skybox_program = gl.program(
    vertex_shader= _vertex_shader_code,
    fragment_shader= _fragment_shader_code
)

skybox_renderable = gl.vertex_array(skybox_program,
                [(gl.buffer(_geom), "2f", "position")],
                index_buffer=gl.buffer(_index),index_element_size=4
            )

### Define Camera Parameters

In [ ]:
displacement_vector = 2*bound.radius*glm.rotate(glm.vec3(0,1,0), glm.radians(85), glm.vec3(1,0,0)) #glm.vec3(0,0,1) 

light_displacement_vector = 2*bound.radius*glm.rotate(glm.vec3(0,1,0), glm.radians(45), glm.vec3(1,0,0)) 
    
target_point = glm.vec3(bound.center)
up_vector = glm.vec3(0,1,0)

### View volume parameters
fov_radian = glm.radians(45) # In radian
aspect = width/height
near = bound.radius
far = 3*bound.radius
perspectiveMatrix = glm.perspective(fov_radian, aspect, near, far)

### Render loop

In [ ]:
running = True
clock = pygame.time.Clock()
alpha = 0
lightAngle = 0

pause = True   # Keyboard key "p" toggles pause/orbit
metal = False  # Keyboard key "m" toggles metal/diffuse
skybox = False # Keyboard key "s" toggles skybox/noSkybox

gl.depth_func = '<=' 
gl.enable(gl.DEPTH_TEST)
while running:   
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif (event.type ==  pygame.KEYDOWN):
            if  event.key == 27:
                running = False
            elif event.key == pygame.K_p:
                pause = not pause
            elif event.key == pygame.K_s:
                skybox = not skybox
            elif event.key == pygame.K_m:
                metal = not metal
            elif event.key == pygame.K_LEFT:
                lightAngle -= 5
            elif event.key == pygame.K_RIGHT:
                lightAngle += 5
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
            perspectiveMatrix = glm.perspective(fov_radian, width/height, near, far)

    # create the aspect ratio correction matrix
    new_displacement_vector = glm.rotate(displacement_vector, glm.radians(alpha), glm.vec3(0,1,0))

    new_light_displacement_vector = glm.rotate(light_displacement_vector, glm.radians(lightAngle), glm.vec3(0,1,0))
    
    eye_point = target_point + new_displacement_vector

    viewMatrix = glm.lookAt(eye_point, target_point, up_vector)

    gl.clear(0.5,0.5,0.0)

    # Render Relfector
    program = model_program
    program["view"].write(viewMatrix)
    program["perspective"].write(perspectiveMatrix)   
    program["eye_position"].write(eye_point)
    program["light"].write(new_light_displacement_vector)
    program["map"] = 0
    gold_sampler.use(0)
    program["metal"] = metal
    render()

    if skybox:# Render Skybox
        program = skybox_program
        skybox_renderable.render()
    
    pygame.display.flip()
    clock.tick(60)  # limits FPS to 10
    if not pause:
        alpha +=  1
        if alpha > 360:
            alpha = 0
    #running = False
pygame.display.quit()